In [1]:
%config IPCompleter.greedy=True

In [3]:

import os
import matplotlib.pyplot as plt                        
import numpy as np

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import Image

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim

Matplotlib is building the font cache using fc-list. This may take a moment.


In [4]:


def myloader(filename):
    return Image.open(filename).convert('RGB')

standard_normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
                        transforms.RandomResizedCrop(224),
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomRotation(10),
                        transforms.ToTensor(),
                        standard_normalization
                        ])


valid_transform = transforms.Compose([
                        transforms.Resize(256),
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        standard_normalization
                        ])


test_transform = transforms.Compose([
                        transforms.Resize(224,224),
                        transforms.ToTensor(),
                        standard_normalization
                        ])

batch_size = 20

image_test = datasets.ImageFolder('../data/test',transform = test_transform,loader=myloader)
test = torch.utils.data.DataLoader(dataset=image_test, batch_size=batch_size)

image_valid = datasets.ImageFolder('../data/valid',transform = valid_transform,loader=myloader)
valid = torch.utils.data.DataLoader(dataset=image_valid, batch_size=batch_size)

image_train = datasets.ImageFolder('../data/train',transform = train_transform, loader=myloader)
train = torch.utils.data.DataLoader(dataset=image_train,batch_size=batch_size ,shuffle=True)

loaders = {'test':test,'valid':valid,'train':train}



In [5]:
## model architecture resnet18 used 
model = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
52.5%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100.0%


In [6]:
# freezing the parameters for the model
# prevents the training the model parameters
for param in model.parameters():
    param.requires_grad = False

# creating a new classifier
classifier = nn.Sequential(
                            nn.Linear(512,200),
                            nn.ReLU(),
                            nn.Dropout(0.3),
                            nn.Linear(200,100),
                            nn.ReLU(),
                            nn.Dropout(0.3),
                            nn.Linear(100,3))

# replacing model classifier
model.fc = classifier

In [7]:
# defining loss function and optimizer
optimizer = optim.SGD(model.fc.parameters(), lr = 0.3)
criterion = nn.CrossEntropyLoss()

In [9]:
# using cuda: transfering model to CUDA for training
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('cuda enabled')
    model = model.cuda()

cuda enabled


In [10]:
def train(epochs,model,loaders,optimizer,criterion,use_cuda,save_path):
    min_valid_loss = np.Inf
    
    for epoch in range(epochs):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx,(data,target) in enumerate(loaders['train']):
            
            optimizer.zero_grad()
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            output = model.forward(data)
            loss = criterion(output, target)
            
            loss.backward()
            
            optimizer.step()
            
            # checks
            if batch_idx == 0:
                print(loss.data)
                print(train_loss)
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if batch_idx % 100 == 0:
                print(f"for epoch:{epoch} batch({batch_idx + 1})....train loss = {train_loss}:.6f")
        
        model.eval()
        for batch_idx,(data,target) in enumerate(loaders['valid']):
            
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                
            output = model(data)
            loss = criterion(output,target)

            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        # saving model in case the validation loss decreases
        if(valid_loss < min_valid_loss):
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            min_valid_loss,
            valid_loss))
            
            torch.save(model.state_dict(),save_path)
            min_valid_loss = valid_loss
    
    return model

In [ ]:
model = train(20,model,loaders,optimizer,criterion,use_cuda,'model_scratch.pt')

tensor(1.1405, device='cuda:0')
0.0
for epoch:0 batch(1)....train loss = 1.1404982805252075:.6f
Epoch: 0 	Training Loss: 0.862800 	Validation Loss: 1.081882
Validation loss decreased (inf --> 1.081882).  Saving model ...
tensor(0.7600, device='cuda:0')
0.0
for epoch:1 batch(1)....train loss = 0.7599705457687378:.6f


In [ ]:


for img_file in os.listdir('../data/valid/nevus'):
    img_path = os.path.join('../data/valid/nevus/', img_file)